In [31]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
torch.manual_seed(0)

import networkx as nx
import pandas as pd

from sklearn.neighbors import KernelDensity

from reinforcement import environment, Policy, reinforce, evaluate
from metrics import causal_edge_score, causal_cfe_from_parent, sparsity_num, sparsity_cat, L1
warnings.filterwarnings("ignore")

### The SCM

In [3]:
SCM = {i: {'input':[], 'func':None} for i in range(12)}
SCM[0] = {'input':[2], 'func': lambda a: a}

In [4]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'

### Train the classifier

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer

In [6]:
clf_data_train = pd.read_csv('./adult_data/adult_data_train.csv')
clf_data_test = pd.read_csv('./adult_data/adult_data_test.csv')

In [7]:
target = clf_data_train["y"]
# Split data into train and test
x_train= clf_data_train.drop("y", axis=1)

categorical = ['workclass', 'marital-status', 'occupation',	'relationship',	'race',	'sex', 'native-country']
numerical = x_train.columns.difference(categorical)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='error'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', MLPClassifier(random_state=0))])
model = clf.fit(x_train, clf_data_train['y'])

In [8]:
np.mean(clf.predict(clf_data_test.drop('y', axis=1)) == clf_data_test['y'])

0.8257793499889454

In [9]:
pred_train = clf.predict(clf_data_train.drop('y', axis=1)).astype('int')
pred_test = clf.predict(clf_data_test.drop('y', axis=1)).astype('int')
cf_data_train = clf_data_train.drop('y', axis=1)[pred_train==0]
cf_data_test = clf_data_test.drop('y', axis=1)[pred_test==0]
scm = SCM

In [10]:
cat_feats = [cf_data_test.columns.get_loc(name) for name in categorical]
train_num_mean = clf_data_train[numerical].mean()
train_num_std = clf_data_train[numerical].std()
train_data_norm = clf_data_train.drop('y', axis=1)
train_data_norm[numerical] = ((train_data_norm[numerical] - train_num_mean)/ train_num_std).to_numpy()

### Setup and training

In [19]:
DONE_THRESHOLD = 0.5
REG_BETA = 0.5
CON_DIST_BETA = 0
ACTION_ENTROPY_BETA = 0.001
N_ALLOWED_STD = 1.5
MAX_STEP = 8
cat_feats_names = categorical
n_classes = [len(np.unique(train_data_norm[name])) for name in cat_feats_names]
disallowed_feats = [cf_data_train.columns.get_loc(name) for name in ['marital-status', 'race', 'sex', 'native-country']]
directions = {0: 1, 2: 1} # age and education cant decrease
policy = Policy(n_feats=11, cat_feats=cat_feats, n_classes=n_classes, disallowed_feats=disallowed_feats, hidden_size=128).to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-4)
env = environment(clf, clf_data_train.drop('y',axis=1).mean().to_numpy(), clf_data_train.drop('y',axis=1).std().to_numpy(),\
     train_data_norm.min().to_numpy(), train_data_norm.max().to_numpy(), cat_feats, list(cf_data_train.columns), \
     scm, DONE_THRESHOLD, REG_BETA, MAX_STEP, disallowd_feats=disallowed_feats, directions=directions, order_matter=False)
# scores = reinforce(cf_data_train, env, policy, optimizer, checkpoint_path='./ckpt_adult1', n_episodes=40000, print_every=1000, con_dist=None, n_allowed_std=N_ALLOWED_STD, CON_DIST_BETA=CON_DIST_BETA, ACTION_ENTROPY_BETA = ACTION_ENTROPY_BETA, device=device)

Episode 1000	Average Score: 0.60
Episode 2000	Average Score: 0.74
Episode 3000	Average Score: 0.62
Episode 4000	Average Score: 0.87
Episode 5000	Average Score: 0.96
Episode 6000	Average Score: 0.85
Episode 7000	Average Score: 1.09
Episode 8000	Average Score: 1.06
Episode 9000	Average Score: 1.27
Episode 10000	Average Score: 1.46
Episode 11000	Average Score: 1.58
Episode 12000	Average Score: 1.74
Episode 13000	Average Score: 2.07
Episode 14000	Average Score: 2.28
Episode 15000	Average Score: 2.87
Episode 16000	Average Score: 3.12
Episode 17000	Average Score: 3.60
Episode 18000	Average Score: 4.00
Episode 19000	Average Score: 4.47
Episode 20000	Average Score: 4.68
Episode 21000	Average Score: 4.88
Episode 22000	Average Score: 4.92
Episode 23000	Average Score: 4.89
Episode 24000	Average Score: 4.96
Episode 25000	Average Score: 4.96
Episode 26000	Average Score: 5.01
Episode 27000	Average Score: 4.99
Episode 28000	Average Score: 5.02
Episode 29000	Average Score: 5.01
Episode 30000	Average S

In [13]:
# torch.save(policy, './adult_policy_best.pt')
with open('./ckpt_adult/best_model.pt', 'rb') as f:
    policy = torch.load(f)

In [14]:
r = evaluate(cf_data_test, env, policy, device=device, print_file='./result.txt')
cfs = pd.DataFrame(r['CF'], columns=cf_data_test.columns)

In [16]:
cfs

,age,workclass,education,marital-status,occupation,relationship,race,sex,capitalloss,hoursperweek,native-country
0,59.0,1.0,15.0,1.0,1.0,1.0,0.0,0.0,0.0,98.0,0.0
1,38.0,2.0,15.0,2.0,4.0,0.0,0.0,1.0,0.0,40.0,0.0
2,26.0,5.0,15.0,0.0,1.0,2.0,0.0,1.0,4356.0,45.0,0.0
3,23.0,2.0,15.0,0.0,4.0,5.0,2.0,0.0,4356.0,30.0,0.0
4,41.0,2.0,15.0,2.0,9.0,0.0,0.0,0.0,0.0,99.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3623,42.0,1.0,15.0,2.0,9.0,4.0,0.0,1.0,0.0,99.0,0.0
3624,90.0,1.0,15.0,0.0,6.0,1.0,0.0,1.0,4356.0,40.0,0.0
3625,56.0,2.0,15.0,1.0,10.0,1.0,0.0,0.0,0.0,66.0,0.0
3626,56.0,3.0,15.0,2.0,1.0,0.0,0.0,0.0,0.0,99.0,0.0


In [22]:
mask = clf.predict(cfs)
mask = [i for i in range(len(mask)) if mask[i]]

In [27]:
print('validity:', np.mean(model.predict(cfs)))
print('sparsity_num: ', sparsity_num(cfs.iloc[mask], cf_data_test.iloc[mask], cat_feats=categorical))
print('sparsity_cat: ', sparsity_cat(cfs.iloc[mask], cf_data_test.iloc[mask], cat_feats=categorical))
print('L1: ', L1(cfs.iloc[mask], cf_data_test.iloc[mask], cat_feats=categorical, stds=clf_data_train.std()))
print('sparsity_num(causal perspective): ', np.mean(np.array(r['reg'])[mask][:,[0, 2, 8, 9]]==0))
print('sparsity_cat(causal perspective): ', np.mean(np.array(r['reg'])[mask][:,[cat_feats]]==0))
print('L1(causal perspective): ', np.mean(np.abs(np.array(r['reg'])[mask][:,[0, 2, 8, 9]])))

validity: 0.9944873208379272
sparsity_num:  0.3668930155210643
sparsity_cat:  0.8535793474817865
L1:  1.8719410238516443
sparsity_num(causal perspective):  0.5922949002217295
sparsity_cat(causal perspective):  0.8535793474817865
L1(causal perspective):  [1.77154609]


In [28]:
# constraints
rule1 = (cfs['age'].iloc[mask].to_numpy() - cf_data_test['age'].iloc[mask].to_numpy()) >= 0
rule2 = (cfs['education'].iloc[mask].to_numpy() - cf_data_test['education'].iloc[mask].to_numpy()) >= 0
rule3 = ((cfs['education'].iloc[mask].to_numpy() - cf_data_test['education'].iloc[mask].to_numpy()) == 0) + (cfs['age'].iloc[mask].to_numpy() > cf_data_test['age'].iloc[mask].to_numpy())*(cfs['education'].iloc[mask].to_numpy() > cf_data_test['education'].iloc[mask].to_numpy())
print(np.mean(rule1*rule2*rule3>0))

0.9986141906873615
